<a href="https://colab.research.google.com/github/mahendrakevin/Speech-Accent-Recognition/blob/master/Speech%20Accent%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%cd /content/drive/MyDrive/Data Science/Speech-Accent-Recognition-master

/content/drive/MyDrive/Data Science/Speech-Accent-Recognition-master


In [3]:
!pip install pydub

In [ ]:
!python fromwebsite.py bio_metadata.csv english spanish arabic

downloading from http://accent.gmu.edu/browse_language.php?function=find&language=english
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=spanish
downloading from http://accent.gmu.edu/browse_language.php?function=find&language=arabic
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=61
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=77
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=88
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=99
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=110
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=121
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=132
downloading from http://accent.gmu.edu/browse_language.php?function=detail&speakerid=143
downloading from http:/

In [ ]:
!python getaudio.py bio_metadata.csv

python3: can't open file 'getaudio.py': [Errno 2] No such file or directory


In [6]:
import pandas as pd
from datetime import datetime
import re

In [9]:
start = datetime.now()
metadata = pd.read_csv("/content/drive/MyDrive/Data Science/Speech-Accent-Recognition-master/bio_metadata.csv")
metadata.head()
end_time = datetime.now()
print("Durasi: {}".format(end_time - start))

Durasi: 0:00:00.015478


In [ ]:
metadata.native_language = metadata.native_language.apply(lambda x: re.sub(r"\([a-z]{3}\)", "", x))
metadata.native_language = metadata.native_language.str.replace("\n", "")
metadata.head()

,href,language_num,sex,birth_place,native_language,other_languages,age_sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence,age
0,http://accent.gmu.edu/browse_language.php?func...,english1,male,"['pennsylvania,', 'usa']",english,"['mandarin', '']","['42,', 'male', '']",0.0,naturalistic,['usa'],41.0,42.0
1,http://accent.gmu.edu/browse_language.php?func...,english2,female,"['birmingham,', 'uk']",english,"['german', 'mandarin', '']","['30,', 'female', '']",0.0,naturalistic,['uk'],30.0,30.0
2,http://accent.gmu.edu/browse_language.php?func...,english3,female,"['brisbane,', 'australia']",english,['none'],"['26,', 'female', '']",0.0,naturalistic,['australia'],26.0,26.0
3,http://accent.gmu.edu/browse_language.php?func...,english4,female,"['bay,', 'jamaica']",english,"['spanish', 'french', '']","['53,', 'female', '']",0.0,naturalistic,"['jamaica,usa']",53.0,53.0
4,http://accent.gmu.edu/browse_language.php?func...,english5,male,"['virginia,', 'usa']",english,['none'],"['62,', 'male', '']",0.0,naturalistic,['usa'],62.0,62.0


In [ ]:
metadata.to_csv("bio_metadata_preprocessed.csv")

In [ ]:
start_time = datetime.now()
!python trainmodel.py bio_metadata_preprocessed.csv accent_model
end_time = datetime.now()
print("Durasi: {}".format(end_time - start_time))

2021-02-05 07:30:05.423039: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Entering main
Loading wav files....
Converting to MFCC....
X_train shape: (48951, 13, 30, 1)
48951 training samples
2021-02-05 08:17:31.168096: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-05 08:17:31.256632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-05 08:17:31.336558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-02-05 08:17:31.338178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB de

In [10]:
import pandas as pd
from collections import Counter
import sys
import getsplit

from keras import utils
import accuracy
import multiprocessing
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import accuracy, getsplit, trainmodel

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import MaxPooling2D, Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import load_model

In [11]:
model = load_model("/content/drive/MyDrive/Data Science/models/accent_model.h5")

In [13]:
df = pd.read_csv("/content/drive/MyDrive/Data Science/Speech-Accent-Recognition-master/bio_metadata_preprocessed.csv")
filtered_df = getsplit.filter_df(df)
print(getsplit.split_people(filtered_df))

[667     spanish22
0        english1
811    spanish166
137    english138
281    english282
          ...    
160    english161
9       english10
250    english251
723     spanish78
815    spanish170
Name: language_num, Length: 1589, dtype: object, 1054     arabic181
739      spanish94
240     english241
188     english189
177     english178
           ...    
163     english164
628     english629
296     english297
91       english92
580     english581
Name: language_num, Length: 398, dtype: object, 667    spanish
0      english
811    spanish
137    english
281    english
        ...   
160    english
9      english
250    english
723    spanish
815    spanish
Name: native_language, Length: 1589, dtype: object, 1054     arabic
739     spanish
240     english
188     english
177     english
         ...   
163     english
628     english
296     english
91      english
580     english
Name: native_language, Length: 398, dtype: object]


In [15]:
start_time = datetime.now()
train_pct_index = np.random.rand(len(filtered_df)) < 0.95
data_test = filtered_df.language_num[~train_pct_index]
y = filtered_df.native_language[~train_pct_index]
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
data_test = pool.map(trainmodel.get_wav, data_test)
data_test = pool.map(trainmodel.to_mfcc, data_test)
end_time = datetime.now()
print("Durasi: {}".format(end_time - start_time))

Durasi: 0:03:02.642297


In [16]:
labels = trainmodel.to_categorical(y)
test, y = trainmodel.make_segments(data_test, labels)

In [18]:
start_time = datetime.now()
y_predicted = accuracy.predict_class_all(trainmodel.create_segmented_mfccs(test), model)
end_time = datetime.now()
print("Durasi: {}".format(end_time - start_time))

Durasi: 0:01:46.973672


In [20]:
print('Confusion matrix of total samples:\n', np.sum(accuracy.confusion_matrix(y_predicted, y),axis=1))
print('Confusion matrix:\n',accuracy.confusion_matrix(y_predicted, y))
print('Accuracy:', accuracy.get_accuracy(y_predicted, y))

Confusion matrix of total samples:
 [ 778 2136  753]
Confusion matrix:
 [[  48  340  390]
 [ 117  482 1537]
 [  79  281  393]]
Accuracy: 0.25170439050995363


In [21]:
data_orig = filtered_df.language_num[~train_pct_index]
y_labs = filtered_df.native_language[~train_pct_index]

In [22]:
df['label_id'] = df['native_language'].factorize()[0]
from io import StringIO
kategori_id_data = df[['native_language', 'label_id']].drop_duplicates().sort_values('label_id')
kategori_to_id = dict(kategori_id_data.values)
id_to_kategori = dict(kategori_id_data[['label_id', 'native_language']].values)

In [24]:
labeled =[]
lab2 = []
prediction = []
for lang, label, predicted in zip(data_orig, y_labs, y_predicted):
    u = lang
    s = label
    l = id_to_kategori[predicted]
    labeled.append(u)
    lab2.append(s)
    prediction.append(l)
    print("Label: '{}'".format(u))
    print("Prediksi: '{}'".format(id_to_kategori[predicted]))
    print("")

Label: 'english14'
Prediksi: 'arabic'

Label: 'english23'
Prediksi: 'arabic'

Label: 'english73'
Prediksi: 'arabic'

Label: 'english86'
Prediksi: 'arabic'

Label: 'english94'
Prediksi: 'arabic'

Label: 'english139'
Prediksi: 'arabic'

Label: 'english147'
Prediksi: 'arabic'

Label: 'english150'
Prediksi: 'arabic'

Label: 'english154'
Prediksi: 'arabic'

Label: 'english166'
Prediksi: 'arabic'

Label: 'english185'
Prediksi: 'arabic'

Label: 'english203'
Prediksi: 'arabic'

Label: 'english207'
Prediksi: 'arabic'

Label: 'english209'
Prediksi: 'arabic'

Label: 'english220'
Prediksi: 'arabic'

Label: 'english237'
Prediksi: 'arabic'

Label: 'english297'
Prediksi: 'arabic'

Label: 'english303'
Prediksi: 'arabic'

Label: 'english351'
Prediksi: 'arabic'

Label: 'english358'
Prediksi: 'spanish'

Label: 'english384'
Prediksi: 'arabic'

Label: 'english415'
Prediksi: 'spanish'

Label: 'english464'
Prediksi: 'arabic'

Label: 'english474'
Prediksi: 'english'

Label: 'english526'
Prediksi: 'arabic'

La

In [25]:
df_hasil = pd.DataFrame({'Name Accent': labeled,
     'labels': lab2,
     'predicted_labels': prediction
    })

In [26]:
df_hasil.head()

,Name Accent,labels,predicted_labels
0,english14,english,arabic
1,english23,english,arabic
2,english73,english,arabic
3,english86,english,arabic
4,english94,english,arabic


In [27]:
df_hasil.to_csv("predicted.csv")